In [152]:
#pip install PdfReader

## Convert PDF to Text

In [138]:
import PyPDF2

def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        combined_text = ""
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text = page.extract_text()
            combined_text += text

        return combined_text

# Usage example
pdf_path = '1_55661.pdf'

# context = extract_text_from_pdf(pdf_path)[3268:7000]
# print(context)


In [82]:
def separate_paragraphs(text):
    paragraphs = text.split("  \n")  # Split text by double line breaks

    # Remove leading and trailing whitespaces from each paragraph
    paragraphs = [paragraph.strip() for paragraph in paragraphs]

    return paragraphs


### Count total paragraph from PDF

In [137]:
full_doc = extract_text_from_pdf(pdf_path)
full_paras = separate_paragraphs(full_doc)
len(full_paras)

283

In [96]:
question = "what are the supported platform for ssh runner?"

### Retrieve the Context from Text 

In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar_text(sentence, text_objects):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on text_objects
    tfidf_matrix = vectorizer.fit_transform(text_objects)

    # Transform the input sentence using the fitted vectorizer
    sentence_vector = vectorizer.transform([sentence])

    # Calculate cosine similarities between the sentence vector and all text objects
    similarities = cosine_similarity(sentence_vector, tfidf_matrix)

    # Find the index of the most similar text object
    most_similar_index = similarities.argmax()

    # Get the most similar text object
    most_similar_text = text_objects[most_similar_index]

    # Check the length of the most similar text object
    if len(most_similar_text) < 2000:
        # Find the indices and similarities of the related text objects
        sorted_indices = similarities.argsort()[0][::-1]
        sorted_similarities = similarities[0, sorted_indices]

        # Iterate over the related text objects and add paragraphs until length exceeds 2000 characters
        for idx, similarity in zip(sorted_indices, sorted_similarities):
            if idx != most_similar_index:
                related_text = text_objects[idx]
                if len(most_similar_text) + len(related_text) + 2 > 2000:  # +2 for "\n\n"
                    break
                most_similar_text += "\n\n" + related_text

    return most_similar_text
question = "what are the supported platform for ssh runner?"

most_relevant_text = find_most_similar_text(question, full_paras)
print("Most Relevant context:", most_relevant_text)
len(most_relevant_text)

Most Relevant context: First, download the gitlab-runner distribution for the appropriate platform at 
https://docs.gitlab.com/runner/install/ . In this configura tion, the SSH GitLab runner is installed on a Linux 
x86 Red Hat distribution, but similar installation and configuration can be performed on other supported 
platforms.

The SSH runners are executed on a supported platform (Windows, Linux or MacOS) and connect to a 
target machine through SSH for the pipeline execution. In that configuration, SSH runners act like 
gateways to connect platforms: the GitLab server will send the pipeline actions to the SSH runner which 
will forward them to the target machine, in this case th e z/OS environment. The different stages of the 
pipeline will be executed on the target z/OS machine and results will be sent back to the GitLab server 
through the same mechanism.

Provide the GitLab server URL and the registration token to register the runner. Then provide a description 
for this runner

1789

### Run LLM model on most_relevant_text 

In [157]:
import os
import time
from textwrap import dedent
from PIL import Image
import json
import re
import pandas as pd
from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from nltk.translate import meteor_score as ms
from rouge_score import rouge_scorer
from bs4 import BeautifulSoup
import requests
from nltk.translate import bleu_score
import nltk
import torch
import gc


#bamToken = 'Bam-key'


# question  = "What is the purpose of the provided extracts and how can they be integrated into a pipeline for z/OS-related capabilities?"

question = "what are the supported platform for ssh runner?"

chat_history = f"Answer the question based on the context below. " + \
    "Context: "  + most_relevant_text + \
    " Question: " + question

model_input = chat_history.replace("<split>", "\n")

print("INPUT PROMPT: ", model_input)


headers = {
    'Content-Type': 'application/json',
    'Authorization': bamToken,
}

json_data = {
    'model_id': 'bigscience/bloom',
    # 'model_id': 'google/flan-ul2',
#     'model_id': 'google/flan-t5-xxl',
    # 'model_id': 'ibm/coga-3b-0.1',
    # 'model_id':'flan-t5-xl-mpt-2zfTOrpU-2023-05-01-19-48-20',
    
    'inputs':  [model_input],

        #bam
        'parameters': {
        # "stream": "true",
        'temperature': 0.3,
        'min_new_tokens': 10,
        'max_new_tokens': 200,
        'stop_sequences': ['Question']

         # Modify this parameter to reduce the batch size
        # 'decoding_method': 'greedy'
        # 'repetition_penalty': 1.0,
    },
}

#bam
# response = requests.post('https://bam-api.res.ibm.com/v1/generate', headers=headers, json=json_data)

#Coga
response = requests.post('https://fmaas-dev-api.bx.cloud9.ibm.com/v1/generate', headers=headers, json=json_data)


json_response = json.loads(response.content.decode("utf-8"))
print("BAM OUTPUT: ", json_response)

model_output1 = json_response['results'][0]['generated_text']
model_output1 = model_output1.replace("Question", '')
model_output1 = model_output1.replace("Answer: ", '')

# Seprate sentences
sentences = model_output1.split(". ")
# remove duplicates SENTENCES
unique_sentences = list( dict.fromkeys(sentences))

if not model_output1.endswith("."):
# remove the last sentence if not . at last
    unique_sentences.pop()

# join unique sentences back into a text 
model_output = ". ".join(unique_sentences)+ "."

INPUT PROMPT:  Answer the question based on the context below. Context: First, download the gitlab-runner distribution for the appropriate platform at 
https://docs.gitlab.com/runner/install/ . In this configura tion, the SSH GitLab runner is installed on a Linux 
x86 Red Hat distribution, but similar installation and configuration can be performed on other supported 
platforms.

The SSH runners are executed on a supported platform (Windows, Linux or MacOS) and connect to a 
target machine through SSH for the pipeline execution. In that configuration, SSH runners act like 
gateways to connect platforms: the GitLab server will send the pipeline actions to the SSH runner which 
will forward them to the target machine, in this case th e z/OS environment. The different stages of the 
pipeline will be executed on the target z/OS machine and results will be sent back to the GitLab server 
through the same mechanism.

Provide the GitLab server URL and the registration token to register the ru

In [158]:
print("FINAL ANSWER: ", model_output1)

FINAL ANSWER:   
Linux, Windows, MacOS 


## Example Questions

In [167]:
question = "the execution of the pipeline is performed through which channel?"

most_relevant_text = find_most_similar_text(question, full_paras)

chat_history = f"Answer the question based on the context below. " + \
    "Context: "  + most_relevant_text + \
    " Question: " + question

model_input = chat_history.replace("<split>", "\n")

print("INPUT PROMPT: ", model_input)


headers = {
    'Content-Type': 'application/json',
    'Authorization': bamToken,
}

json_data = {
    'model_id': 'bigscience/bloom',

    'inputs':  [model_input],

        #bam
        'parameters': {
        # "stream": "true",
        'temperature': 0.3,
        'min_new_tokens': 10,
        'max_new_tokens': 200,
        'stop_sequences': ['Question']
    },
}

#bam
# response = requests.post('https://bam-api.res.ibm.com/v1/generate', headers=headers, json=json_data)

#Coga
response = requests.post('https://fmaas-dev-api.bx.cloud9.ibm.com/v1/generate', headers=headers, json=json_data)


json_response = json.loads(response.content.decode("utf-8"))
print("BAM OUTPUT: ", json_response)

model_output1 = json_response['results'][0]['generated_text']
model_output1 = model_output1.replace("Question", '')
model_output1 = model_output1.replace("Answer: ", '')

# Seprate sentences
sentences = model_output1.split(". ")
# remove duplicates SENTENCES
unique_sentences = list( dict.fromkeys(sentences))

if not model_output1.endswith("."):
# remove the last sentence if not . at last
    unique_sentences.pop()

# join unique sentences back into a text 
model_output = ". ".join(unique_sentences)+ "."

INPUT PROMPT:  Answer the question based on the context below. Context: As the execution of the pipeline is performed through an SSH channel, it is important to have the Bash 
shell available f or the user who will execute the build actions. More importantly, the Bash shell must be 
set to be the first program executed by the user when logging into z/OS Unix System Services. This 
parameter is controlled through the PROGRAM keyword in the RACF’s us er definition, as part of the OMVS 
segment. To change the PROGRAM value for an existing user, please customize and use the following 
RACF ALTUSER command:

At the end of the job execution, the developer can open the Wazi Analyze UI, with the link provided in the 
output log of the job. Through this web -based  solution, analysis can be performed with the latest versions 
of the artifacts that were used during the build, ensuring consistency between the structure of the 
application and the actual load modules that were built.

Using the mu

In [168]:
print("FINAL ANSWER: ", model_output1)

FINAL ANSWER:   SSH channel. : it is important to have the Bash shell available for the user who will execute the build actions. Yes. 


In [171]:
question = "the execution of the pipeline is performed through which channel?"

most_relevant_text = find_most_similar_text(question, full_paras)

chat_history = f"Answer the question based on the context below. " + \
    "Context: "  + most_relevant_text + \
    " Question: " + question

model_input = chat_history.replace("<split>", "\n")

print("INPUT PROMPT: ", model_input)


headers = {
    'Content-Type': 'application/json',
    'Authorization': bamToken,
}

json_data = {
    'model_id': 'bigscience/bloom',    
    'inputs':  [model_input],

        #bam
        'parameters': {
        # "stream": "true",
        'temperature': 0.3,
        'min_new_tokens': 10,
        'max_new_tokens': 200,
        'stop_sequences': ['Question']
    },
}

#bam
# response = requests.post('https://bam-api.res.ibm.com/v1/generate', headers=headers, json=json_data)

#Coga
response = requests.post('https://fmaas-dev-api.bx.cloud9.ibm.com/v1/generate', headers=headers, json=json_data)


json_response = json.loads(response.content.decode("utf-8"))
print("BAM OUTPUT: ", json_response)

model_output1 = json_response['results'][0]['generated_text']
model_output1 = model_output1.replace("Question", '')
model_output1 = model_output1.replace("Answer: ", '')

# Seprate sentences
sentences = model_output1.split(". ")
# remove duplicates SENTENCES
unique_sentences = list( dict.fromkeys(sentences))

if not model_output1.endswith("."):
# remove the last sentence if not . at last
    unique_sentences.pop()

# join unique sentences back into a text 
model_output = ". ".join(unique_sentences)+ "."

print("FINAL ANSWER: ", model_output1)

INPUT PROMPT:  Answer the question based on the context below. Context: As the execution of the pipeline is performed through an SSH channel, it is important to have the Bash 
shell available f or the user who will execute the build actions. More importantly, the Bash shell must be 
set to be the first program executed by the user when logging into z/OS Unix System Services. This 
parameter is controlled through the PROGRAM keyword in the RACF’s us er definition, as part of the OMVS 
segment. To change the PROGRAM value for an existing user, please customize and use the following 
RACF ALTUSER command:

At the end of the job execution, the developer can open the Wazi Analyze UI, with the link provided in the 
output log of the job. Through this web -based  solution, analysis can be performed with the latest versions 
of the artifacts that were used during the build, ensuring consistency between the structure of the 
application and the actual load modules that were built.

Using the mu

In [185]:
question = "Which variable controls GitLab CI/CD Git checkout?"

most_relevant_text = find_most_similar_text(question, full_paras)

chat_history = f"Answer the question based on the context below. " + \
    "Context: "  + most_relevant_text + \
    " Question: " + question

model_input = chat_history.replace("<split>", "\n")

print("INPUT PROMPT: ", model_input)


headers = {
    'Content-Type': 'application/json',
    'Authorization': bamToken,
}

json_data = {
    'model_id': 'bigscience/bloom',
    'inputs':  [model_input],
        'parameters': {
        # "stream": "true",
        'temperature': 0.3,
        'min_new_tokens': 10,
        'max_new_tokens': 200,
        'stop_sequences': ['Question']
    },
}

#bam
# response = requests.post('https://bam-api.res.ibm.com/v1/generate', headers=headers, json=json_data)

#Coga
response = requests.post('https://fmaas-dev-api.bx.cloud9.ibm.com/v1/generate', headers=headers, json=json_data)


json_response = json.loads(response.content.decode("utf-8"))
print("BAM OUTPUT: ", json_response)

model_output1 = json_response['results'][0]['generated_text']
model_output1 = model_output1.replace("Question", '')
model_output1 = model_output1.replace("Answer: ", '')

# Seprate sentences
sentences = model_output1.split(". ")
# remove duplicates SENTENCES
unique_sentences = list( dict.fromkeys(sentences))

if not model_output1.endswith("."):
# remove the last sentence if not . at last
    unique_sentences.pop()

# join unique sentences back into a text 
model_output = ". ".join(unique_sentences)+ "."

print("FINAL ANSWER: ", model_output1)

INPUT PROMPT:  Answer the question based on the context below. Context: To change this default behavior of checking Git for every job, specific variables can be set either at the 
pipeline level or at the job level. The latter option is the preferred way for fine -tuning and manage the 
repository checkout granularly.  The GIT_ST RATEGY12 variable controls how the GitLab CI/CD pipeline 
performs the Git checkout: when set to clone , the entire repository will be cloned for this job; when set 
to fetch , only the changes in the Git repository of the project are retrieved. Whe n set to none , no Git 
operation is performed, and the project’s workspace is left untouched.

Integrating IBM z/OS platform in CI/CD pipelines with GitLab   Page 7/22 3 Setting up an initial pipeline with  GitLab CI /CD 
Once the GitLab SSH runner is set up for z/OS, CI /CD pipelines can be executed on Mainframe. The 
definition of pipelines in GitLab CI are done throug h a configuration file, called .gitlab-ci.y